# Statistical models in R
This notebook is covering:
1. VECM

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

In [20]:
%%R
source("vecm.r")
hub1_name <- "the"
hub2_name <- "nbp"
validation_size <- 250
test_size <- 250
window_size <- 5

In [21]:
%%R
vecm_output <- vecm_system(hub1_name, hub2_name, validation_size, test_size, window_size, verbose=TRUE, save=TRUE)

[1] "Selected number of lags: 4"
[1] "Pair: the | nbp"
[1] "Window Size: 5"
[1] "Test Size: 250"
[1] "the: Mean Absolute Error: 2.014"
[1] "the: Root Mean Squared Error: 2.66"
[1] "nbp: Mean Absolute Error: 2.051"
[1] "nbp: Root Mean Squared Error: 2.888"


In [22]:
%%R
training_model <- vecm_training_model(hub1_name, hub2_name, test_size, window_size)
summary(training_model)

#############
###Model VECM 
#############
Full sample size: 1290 	End sample size: 1285
Number of variables: 2 	Number of estimated slope parameters 20
AIC 7262.757 	BIC 7371.086 	SSR 84276.14
Cointegrating vector (estimated by ML):
   hub1      hub2
r1    1 -1.457273


              ECT               Intercept          hub1 -1          
Equation hub1 0.0170(0.0089).   0.2104(0.1856)     0.2479(0.0547)***
Equation hub2 0.0362(0.0097)*** 0.4268(0.2020)*    0.2747(0.0596)***
              hub2 -1             hub1 -2            hub2 -2           
Equation hub1 -0.0718(0.0505)     -0.1264(0.0551)*   0.0136(0.0506)    
Equation hub2 -0.1280(0.0550)*    -0.2128(0.0599)*** 0.0692(0.0551)    
              hub1 -3            hub2 -3           hub1 -4          
Equation hub1 -0.2530(0.0553)*** 0.2269(0.0505)*** 0.1581(0.0549)** 
Equation hub2 -0.2398(0.0602)*** 0.2121(0.0549)*** 0.2240(0.0597)***
              hub2 -4            
Equation hub1 -0.0102(0.0505)    
Equation hub2 -0.0964(0.0549).

In [23]:
%%R
hub1_predictions <- vecm_output$predictions$hub1
hub1_actuals <- vecm_output$actuals$hub1
hub2_predictions <- vecm_output$predictions$hub2
hub2_actuals <- vecm_output$actuals$hub2


In [24]:
%%R
hub1_forecast_comparison <- data.frame(
  Actual = hub1_actuals,
  Forecasted = hub1_predictions
)

In [25]:
%R -o hub1_forecast_comparison
%R -o hub1_name

hub1_forecast_comparison = hub1_forecast_comparison

actual_trace = go.Scatter(
    x=hub1_forecast_comparison.index,
    y=hub1_forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=hub1_forecast_comparison.index,
    y=hub1_forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title= hub1_name.item() + ': VECM Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()


In [26]:
%%R
hub2_forecast_comparison <- data.frame(
  Actual = hub2_actuals,
  Forecasted = hub2_predictions
)

In [27]:
%R -o hub2_forecast_comparison
%R -o hub2_name

hub2_forecast_comparison = hub2_forecast_comparison

actual_trace = go.Scatter(
    x=hub2_forecast_comparison.index,
    y=hub2_forecast_comparison['Actual'],
    mode='lines',
    name='Actual Prices'
)

forecasted_trace = go.Scatter(
    x=hub2_forecast_comparison.index,
    y=hub2_forecast_comparison['Forecasted'],
    mode='lines',
    name='Forecasted Prices'
)

# Create the plot
layout = go.Layout(
    title= hub2_name.item() + ': VECM Actual vs Forecasted Prices',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Price'),
)

fig = go.Figure(data=[actual_trace, forecasted_trace], layout=layout)
fig.show()
